### Про метрику

Важной частью этого чекпоинта является выбор метрики. Напомним, что на текущий момент видение проекта - LLM / Agent-based ассистент для задач ментального здоровья. То есть в качестве стандартных NLP-метрик необходимо использовать метрики для задачи Language Modeling / текстовой генерации, такие как вывод обученной reward-модели, perplexity, side-by-side сравнение на асессорах, и так далее. При этом для ML-бейзлайна на текущем этапе мы будем решать задачу текстовой классификации. Для этой задачи мы можем использовать классические метрики для текстовой классификации, такие как accuracy, precision, recall, F1-score, и так далее. При этом у нас нет негативых сэмплов, так как каждое наблюдение принадлежит какому-то классу, связанного с ментальным здоровьем. Поэтому будем использовать просто accuracy (сбалансированную по классам, так как датасет является не супер сбалансированным)

In [1]:
import pandas as pd
import numpy as np

In [2]:
# нанов в процентах немного, а данных много, поэтому позволим себе просто удалить наны на текущем этапе
reddit_df = pd.read_csv('../data/reddit_mental_health_posts_preprocessed.csv')
reddit_df = reddit_df[['title', 'body', 'subreddit']]
reddit_df = reddit_df.rename(columns={'subreddit': 'target'})
print('len of reddit_df: ', len(reddit_df))
reddit_df = reddit_df.dropna().sample(10000)
print('len of reddit_df after dropna: ', len(reddit_df))
reddit_df.head(5)

len of reddit_df:  151288
len of reddit_df after dropna:  3000


,title,body,target
125992,afraid dont love girlfriend,deleted,OCD
130950,every single dream kind mental game,“ dream ” last night romanticizing ex boyfrien...,ptsd
123732,zoloft worked relatively well id like try some...,deleted,OCD
132932,high anxiety day came nowhere wth,removed,ptsd
57854,69 high functioning autistic adolescent want r...,deleted,aspergers


In [3]:
# видно, что датасет не супер сбалансирован
reddit_df.target.value_counts()

target
OCD           837
ADHD          729
ptsd          494
depression    477
aspergers     463
Name: count, dtype: int64

### Соединим title и body, и построим эмбеддинги

Для сравнения будем использовать:

+ tf-idf

+ tf-idf + SVD разложение (понижение размерности)

+ Word2Vec с усреднением эмбеддингов по словам

+ Простенький трансформер

In [4]:
reddit_df['text'] = 'Title: ' + reddit_df['title'] + '; Body: ' + reddit_df['body']
reddit_df = reddit_df.drop(['title', 'body'], axis=1)
reddit_df.head(5)

,target,text
125992,OCD,Title: afraid dont love girlfriend; Body: deleted
130950,ptsd,Title: every single dream kind mental game; Bo...
123732,OCD,Title: zoloft worked relatively well id like t...
132932,ptsd,Title: high anxiety day came nowhere wth; Body...
57854,aspergers,Title: 69 high functioning autistic adolescent...


In [5]:
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sentence_transformers import SentenceTransformer
from gensim.models import Word2Vec

In [6]:
def create_embeddings(df):
    results = []
    df1 = df.copy()
    df2 = df.copy()
    df3 = df.copy()
    df4 = df.copy()
    # Let's start with tf-idf embeddings
    print("Creating tf-idf embeddings")
    tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    X_tfidf = tfidf_vectorizer.fit_transform(df1['text']).toarray()
    results.append((df1, "tf-idf", X_tfidf))
    
    # Now let's add SVD to serve as dimensionality reduction
    print("Creating tf-idf + SVD embeddings")
    svd = TruncatedSVD(n_components=300, random_state=42)
    X_tfidf_svd = svd.fit_transform(X_tfidf)
    results.append((df2, "tf-idf+SVD", X_tfidf_svd))
    
    # Time to do something more advanced - shall we use transformers?
    print("Creating Sentence-BERT embeddings")
    model = SentenceTransformer('all-MiniLM-L6-v2') # something lightweight so that the laptop does not die
    sentences = df3['text'].tolist()
    X_sbert = model.encode(sentences, batch_size=32, show_progress_bar=True)
    results.append((df3, "Sentence-BERT", X_sbert))
    
    # Word2Vec with averaging is not dead, is it?
    print("Creating Word2Vec embeddings")
    tokenized_texts = [word_tokenize(text.lower()) for text in df4['text']]
    w2v_model = Word2Vec(
        sentences=tokenized_texts,
        vector_size=128,
        window=5,
        min_count=5,
        workers=4,
        seed=42
    )
    # Averaging word vectors
    doc_vectors = []
    for doc in tokenized_texts:
        word_vecs = [w2v_model.wv[word] for word in doc if word in w2v_model.wv]
        if len(word_vecs) > 0:
            doc_vectors.append(np.mean(word_vecs, axis=0))
        else:
            doc_vectors.append(np.zeros(w2v_model.vector_size))
    X_word2vec = np.array(doc_vectors)
    results.append((df4, "Word2Vec", X_word2vec))
    
    print('Success')
    return results

In [7]:
embedding_df = create_embeddings(reddit_df)

Creating tf-idf embeddings
Creating tf-idf + SVD embeddings
Creating Sentence-BERT embeddings


Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Creating Word2Vec embeddings
Success


### Начнем моделировать. Для каждого из эмбеддингов построим 4 модели:

+ Logistic Regression

+ KNN

+ CatBoost

+ MLP (простой MLP из sklearn)

После этого измерим качество моделей на тестовой выборке и выберем лучшую. Будем выбирать по balanced accuracy, а выборку будем делить на трейн и тест в соотношении 90:10 (для CatBoost и MLP будем использовать дополнительную валидационную выборку)

In [20]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, confusion_matrix

### CatBoost GridSearch

In [29]:
results = []
for max_depth in range(3, 8):
    for iterations in range(50, 1051, 100):
        model = CatBoostClassifier(
            iterations=iterations,
            max_depth=max_depth,
            random_state=42,
            verbose=False,
            early_stopping_rounds=20
        )
        
        model.fit(X_train, y_train, eval_set=(X_val, y_val))
        
        y_pred = model.predict(X_val)
        
        accuracy = round(balanced_accuracy_score(y_val, y_pred), 3)
        precision = round(precision_score(y_val, y_pred, average='micro'), 3)
        recall = round(recall_score(y_val, y_pred, average='micro'), 3)
        
        results.append(
            ((iterations, max_depth), (accuracy, precision, recall))
        )
        
        print((iterations, max_depth), (accuracy, precision, recall))

(50, 3) (0.804, 0.809, 0.809)
(150, 3) (0.908, 0.914, 0.914)
(250, 3) (0.953, 0.957, 0.957)
(350, 3) (0.952, 0.957, 0.957)
(450, 3) (0.965, 0.969, 0.969)
(550, 3) (0.973, 0.975, 0.975)
(650, 3) (0.977, 0.981, 0.981)
(750, 3) (0.985, 0.988, 0.988)
(850, 3) (0.977, 0.981, 0.981)
(950, 3) (0.972, 0.975, 0.975)
(1050, 3) (0.98, 0.981, 0.981)
(50, 4) (0.868, 0.87, 0.87)
(150, 4) (0.965, 0.969, 0.969)
(250, 4) (0.973, 0.975, 0.975)
(350, 4) (0.992, 0.994, 0.994)
(450, 4) (0.992, 0.994, 0.994)
(550, 4) (0.977, 0.981, 0.981)
(650, 4) (0.984, 0.988, 0.988)
(750, 4) (0.984, 0.988, 0.988)
(850, 4) (0.98, 0.981, 0.981)
(950, 4) (0.984, 0.988, 0.988)
(1050, 4) (0.984, 0.988, 0.988)
(50, 5) (0.913, 0.92, 0.92)
(150, 5) (0.988, 0.988, 0.988)
(250, 5) (0.984, 0.988, 0.988)
(350, 5) (0.972, 0.975, 0.975)
(450, 5) (0.984, 0.988, 0.988)
(550, 5) (0.984, 0.988, 0.988)
(650, 5) (0.98, 0.981, 0.981)
(750, 5) (0.984, 0.988, 0.988)
(850, 5) (0.984, 0.988, 0.988)
(950, 5) (0.984, 0.988, 0.988)
(1050, 5) (0.984

In [32]:
sorted(results, key=lambda val: val[1][2])[-3:]

[((350, 4), (0.992, 0.994, 0.994)),
 ((450, 4), (0.992, 0.994, 0.994)),
 ((150, 6), (0.992, 0.994, 0.994))]

#### Итого, лучшие гиперпараметры - 150 деревьев глубиной до 6

P.S. Отбор происходил по recall, потому что в нашей задаче будто бы цена FN ошибки намного больше, чем FP. Лучше человека случайно причислить к больным и отправить на доп. обследования, чем объявить здоровым несмотря на болезнь. 

In [37]:
def train_and_evaluate_models(X_train, X_test, y_train, y_test, embedding_name, model_name, 
                             X_val=None, y_val=None):
    
    if model_name == "Logistic Regression":
        model = LogisticRegression(random_state=42, max_iter=1000)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
    elif model_name == "CatBoost":
        if X_val is not None and y_val is not None:
            model = CatBoostClassifier(
                iterations=500,
                max_depth=7,
                random_state=42,
                verbose=False,
                early_stopping_rounds=20
            )
            model.fit(X_train, y_train, eval_set=(X_val, y_val))
        else:
            model = CatBoostClassifier(random_state=42, verbose=False)
            model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
    elif model_name == "KNN":
        model = KNeighborsClassifier(n_neighbors=16) # 2 ** 4
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
    elif model_name == "MLP":
        model = MLPClassifier(
            hidden_layer_sizes=(128, 64),
            random_state=42,
            early_stopping=True,
            validation_fraction=0.05,
            max_iter=100
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    accuracy = balanced_accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='micro')
    recall = recall_score(y_test, y_pred, average='micro')
    conf_matrix = confusion_matrix(y_test, y_pred)
    return accuracy, precision, recall, conf_matrix

In [38]:
models = ["Logistic Regression", "CatBoost", "KNN", "MLP"]
results = []

for df_copy, embedding_name, X_embeddings in embedding_df:
    if embedding_name == "tf-idf": # неудачная размерность 5000, все слишком долго обучалось и я решил выкинуть обычный tf-idf из анализа
        continue
    print(f"Processing {embedding_name}")
    y = df_copy['target'].values
    # Везде будет одинаковая тестовая выборка даже при условии, что валидационную использует по сути только CatBoost
    # А нейронка делает валидационную саму из трейновой выборке. Это не супер корректно, но мб кто-то это когда-то поправит :)
    # Но в целом и так не критично, у нас же бейзлайн
    X_temp, X_test, y_temp, y_test = train_test_split(
        X_embeddings, y, test_size=0.1, random_state=42, stratify=y
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.06, random_state=42, stratify=y_temp
    )
    print(f"Train shape: {X_train.shape}, Val shape: {X_val.shape}, Test shape: {X_test.shape}")
    
    for model_name in models:
        print("="*30)
        print(f"Training {model_name}")
        if model_name in ["CatBoost"]:
            accuracy, precision, recall, conf_matrix = train_and_evaluate_models(
                X_train, X_test, y_train, y_test, embedding_name, model_name, X_val, y_val
            )
        else:
            X_train = np.concatenate([X_train, X_val], axis=0)
            y_train = np.concatenate([y_train, y_val], axis=0) # объединяем трейн и валидацию для всех моделей кроме СB
            accuracy, precision, recall, conf_matrix = train_and_evaluate_models(
                X_train, X_test, y_train, y_test, embedding_name, model_name
            )
        results.append({
            'Embedding': embedding_name,
            'Model': model_name,
            'Balanced_Accuracy': accuracy,
            'Micro_Precision': precision,
            'Micro_Recall': recall,
            'Confusion_Matrix': conf_matrix
        })
        print(f"{embedding_name} + {model_name}")
        print(f"Acc: {accuracy:.4f}, Pr:{precision:.4f}, Rec:{recall:.4f}")
        print(conf_matrix)

Processing tf-idf+SVD
Train shape: (2538, 300), Val shape: (162, 300), Test shape: (300, 300)
Training Logistic Regression
tf-idf+SVD + Logistic Regression
Acc: 0.6180, Pr:0.6467, Rec:0.6467
[[53  4  7  6  3]
 [ 7 65  5  4  3]
 [13  6 21  4  2]
 [ 6  7  2 26  7]
 [ 7  5  4  4 29]]
Training CatBoost
tf-idf+SVD + CatBoost
Acc: 0.6327, Pr:0.6533, Rec:0.6533
[[50  6  7  6  4]
 [ 3 64  5  7  5]
 [13  1 23  7  2]
 [10  3  2 30  3]
 [ 6  4  3  7 29]]
Training KNN


AttributeError: 'NoneType' object has no attribute 'split'

In [25]:
results_df = pd.DataFrame(results)
pivot_results = results_df.pivot_table(
        index='Embedding', 
        columns='Model', 
        values='Balanced_Accuracy'
    )

In [26]:
pivot_results

Model,CatBoost,KNN,Logistic Regression,MLP
Embedding,,,,
Sentence-BERT,0.749341,0.728169,0.747167,0.762219
Word2Vec,0.690497,0.597548,0.684865,0.713112
tf-idf+SVD,0.743215,0.583596,0.745709,0.756078


In [28]:
pivot_results.to_csv('reddit_baseline_results_balanced_accuracy.csv')

### Выводы

+ MLP (на удивление) оказалась и самой сильной, и самой устойчивой моделью. В отличие от всех других моделей, здесь качество (в виде balanced accuracy) на тестовой выборке не опустилось ниже 0.7

+ KNN проигрывает всем моделям на всех эмбеддингах, единственный показал качество ниже 0.6. Это значительно хуже других моделей

+ Он же (KNN) оказался самым неустойчивым, где разброс между лучшей и худшей моделью превышает 0.1 (во всех остальных моделях спред в районе 0.05)

+ Несмотря на кардинально разные алгоритмы и структуры моделей, CatBoost и LogReg ведут себя удивительно схоже с точки зрения качества

+ Хоть мы и обучили немного всего, это честный труд :) А еще мы получили на удивление действительно интересные результаты

### Возможный (и желательный) TODO:

+ DONE - Пофиксить логику с валидационной выборкой. Пусть она создается не везде, а только там, где нужно, чтобы не терять данные

+ DONE - Измерить больше метрик, построить confusion матрицы

+ Выбрать лучшую модель (например, CatBoost / MLP на Sentence-BERT эмбеддингах), потюнить ее гиперпараметры, и посмотреть, как будет меняться качество. Выбрать лучшие гиперпараметры, и замерить финальное качество